<a href="https://colab.research.google.com/github/lblum95/AML/blob/master/task3/keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')

Mounted at /content/drive


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.metrics import Precision, Recall
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
import numpy as np
from sklearn.decomposition import KernelPCA, PCA
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import StackingClassifier, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.feature_selection import SelectKBest, VarianceThreshold, chi2, f_classif, mutual_info_classif, \
    SelectFromModel
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from sklearn.preprocessing import OneHotEncoder

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
x_train = pd.read_csv("data/X_train_features_scaled.csv", index_col=0, header=0, low_memory=False)
y_train = pd.read_csv("data/y_train.csv", index_col=0, header=0)
x_test = pd.read_csv("data/X_test_features.csv", index_col=0, header=0, low_memory=False)
normal_features=x_train.iloc[:,:57]
#normal_features=x_train
print(normal_features.shape)
normal_features=normal_features[y_train['y']<3]
print(normal_features.shape)
imputer = SimpleImputer()

scaler = StandardScaler()
#feature_sel = SelectFromModel(RandomForestClassifier())
pca = KernelPCA(kernel='rbf', random_state=42)
features=imputer.fit_transform(normal_features)
selected=scaler.fit_transform(features)
y_tr=y_train[y_train['y']<3]
#selected=feature_sel.fit_transform(feats,y_tr)
#selected=pca.fit_transform(selected)

(5117, 57)
(4947, 57)


In [5]:
from sklearn.model_selection import cross_val_score
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from sklearn.preprocessing import OneHotEncoder
input_dim=selected.shape[1]
print(input_dim)
y_tr=y_train[y_train['y']<3]
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_tr)

57


OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=True)

In [63]:
# define baseline model
def get_model():
	# create model
  model = Sequential()
  model.add(Dropout(0.2))
  model.add(Dense(input_dim, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(input_dim/2, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(input_dim/4, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(3, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[Precision(), Recall()])
  return model

In [64]:
skf=StratifiedKFold(n_splits=5)
for train_index,test_index in skf.split(selected,y_tr):
  X_train = selected[train_index]
  X_test = selected[test_index]
  Y_train = enc.transform(y_tr.iloc[train_index]).toarray()
  Y_test = y_tr.iloc[test_index]
  model = get_model()
  early = EarlyStopping(monitor="val_loss", mode="auto", patience=20, verbose=1)
  redonplat = ReduceLROnPlateau(monitor="val_loss", mode="auto", patience=10, verbose=2)
  callbacks_list = [early, redonplat]  # early

  model.fit(X_train, Y_train, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
  Y_pred=enc.inverse_transform(model.predict(X_test))
  print('Score')
  print(f1_score(Y_test, Y_pred, average='micro'))
  print()
  print()


Epoch 1/1000
112/112 - 1s - loss: 0.9530 - precision_44: 0.6298 - recall_44: 0.3482 - val_loss: 0.6885 - val_precision_44: 0.7730 - val_recall_44: 0.5505
Epoch 2/1000
112/112 - 0s - loss: 0.7519 - precision_44: 0.7424 - recall_44: 0.5229 - val_loss: 0.6295 - val_precision_44: 0.7786 - val_recall_44: 0.5505
Epoch 3/1000
112/112 - 0s - loss: 0.6982 - precision_44: 0.7509 - recall_44: 0.5571 - val_loss: 0.5785 - val_precision_44: 0.8058 - val_recall_44: 0.6288
Epoch 4/1000
112/112 - 0s - loss: 0.6576 - precision_44: 0.7551 - recall_44: 0.6043 - val_loss: 0.5706 - val_precision_44: 0.7864 - val_recall_44: 0.6414
Epoch 5/1000
112/112 - 0s - loss: 0.6389 - precision_44: 0.7655 - recall_44: 0.6223 - val_loss: 0.5697 - val_precision_44: 0.7729 - val_recall_44: 0.6616
Epoch 6/1000
112/112 - 0s - loss: 0.6409 - precision_44: 0.7582 - recall_44: 0.6198 - val_loss: 0.5592 - val_precision_44: 0.7784 - val_recall_44: 0.6742
Epoch 7/1000
112/112 - 0s - loss: 0.6237 - precision_44: 0.7676 - recall_44:

In [10]:
# define baseline model
def get_bottle():
	# create model
  model = Sequential()
  model.add(Dropout(0.15))
  model.add(Dense(input_dim, activation='relu'))
  model.add(Dropout(0.15))
  model.add(Dense(input_dim, activation='relu'))
  model.add(Dropout(0.15))
  model.add(Dense(15, activation='relu'))
  model.add(Dropout(0.15))
  model.add(Dense(input_dim, activation='relu'))
  model.add(Dropout(0.15))
  model.add(Dense(3, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[Precision(), Recall()])
  return model

skf=StratifiedKFold(n_splits=5)
for train_index,test_index in skf.split(selected,y_tr):
  X_train = selected[train_index]
  X_test = selected[test_index]
  Y_train = enc.transform(y_tr.iloc[train_index]).toarray()
  Y_test = y_tr.iloc[test_index]
  model = get_bottle()
  early = EarlyStopping(monitor="val_loss", mode="auto", patience=20, verbose=1)
  redonplat = ReduceLROnPlateau(monitor="val_loss", mode="auto", patience=10, verbose=2)
  callbacks_list = [early, redonplat]  # early

  model.fit(X_train, Y_train, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
  Y_pred=enc.inverse_transform(model.predict(X_test))
  print('Score')
  print(f1_score(Y_test, Y_pred, average='micro'))
  print()
  print()

Epoch 1/1000
112/112 - 1s - loss: 0.8652 - precision_20: 0.6900 - recall_20: 0.4007 - val_loss: 0.6697 - val_precision_20: 0.7614 - val_recall_20: 0.5480
Epoch 2/1000
112/112 - 0s - loss: 0.6875 - precision_20: 0.7510 - recall_20: 0.5819 - val_loss: 0.5909 - val_precision_20: 0.7685 - val_recall_20: 0.6288
Epoch 3/1000
112/112 - 0s - loss: 0.6453 - precision_20: 0.7511 - recall_20: 0.6363 - val_loss: 0.5688 - val_precision_20: 0.7535 - val_recall_20: 0.6717
Epoch 4/1000
112/112 - 0s - loss: 0.6210 - precision_20: 0.7587 - recall_20: 0.6523 - val_loss: 0.5611 - val_precision_20: 0.7784 - val_recall_20: 0.6919
Epoch 5/1000
112/112 - 0s - loss: 0.5994 - precision_20: 0.7634 - recall_20: 0.6731 - val_loss: 0.5644 - val_precision_20: 0.7759 - val_recall_20: 0.6995
Epoch 6/1000
112/112 - 0s - loss: 0.5950 - precision_20: 0.7643 - recall_20: 0.6776 - val_loss: 0.5345 - val_precision_20: 0.7741 - val_recall_20: 0.7096
Epoch 7/1000
112/112 - 0s - loss: 0.5827 - precision_20: 0.7689 - recall_20: